In [20]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofmonth, month
from pyspark.sql.functions import col, max

In [28]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .parquet('yellow_tripdata_2024-10.parquet')

In [29]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [10]:
df.repartition(4).write.parquet("q1_output")

In [11]:
spark.version

'3.5.5'

In [18]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [17]:
oct15_count = df.filter((dayofmonth("tpep_pickup_datetime") == 15) & 
                          (month("tpep_pickup_datetime") == 10)).count()

print("Number of rows with October 15th:", oct15_count)

Number of rows with October 15th: 128893


In [22]:
from pyspark.sql.functions import unix_timestamp, col, max as max_

# Create a new column 'diff_hours' representing the difference in hours
df_diff = df.withColumn("diff_hours", (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 3600.0)

# Compute the maximum difference in hours
max_diff = df_diff.agg(max_("diff_hours").alias("max_diff_hours")).collect()[0]["max_diff_hours"]

print("Biggest difference in hours:", max_diff)

Biggest difference in hours: 162.61777777777777


In [31]:
df1 = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')
df1

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [32]:
from pyspark.sql.functions import col, count

# Cast df1's LocationID to integer to match df's PULocationID
df1_cast = df1.withColumn("LocationID_int", col("LocationID").cast("int"))

# Join the two dataframes on pickup location ID
joined_df = df.join(df1_cast, df["PULocationID"] == df1_cast["LocationID_int"])

# Group by the Zone and count the occurrences
zone_counts = joined_df.groupBy("Zone").agg(count("*").alias("ride_count"))

# Order by ride_count in ascending order and get the least frequent zone
least_frequent_zone = zone_counts.orderBy("ride_count").limit(1)

least_frequent_zone.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `PULocationID` cannot be resolved. Did you mean one of the following? [`LocationID`, `Borough`, `Zone`, `service_zone`].